StudentName : คุณานนต์ กลิ่นจันทร์หอม Rank 30

ID : 6410412015

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import Data & Python Packages

In [ ]:
import pandas as pd
import numpy as np


#import model libraries
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

#import model optimize funcition
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline

#import model optimize funcition
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#visualisation the data
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.simplefilter(action='ignore')
%matplotlib inline

In [ ]:
df_train = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
df_train.head()

In [ ]:
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')
df_test.head()

# 2.Data Cleansing


การทำความสะอาดข้อมูลเป็นสิ่งสำคัญ การทำความสะอาดข้อมูล หรือการทำข้อมูลให้สมบูรณ์ เป็นกระบวนการตรวจสอบและการแก้ไข รายการข้อมูลที่ไม่ถูกต้องออกไปจากชุดข้อมูล ตารางหรือฐานข้อมูล ซึ่งเป็นหลักสำคัญของฐานข้อมูล เพราะหมายถึงความไม่สมบูรณ์ ความไม่ถูกต้อง ความไม่สัมพันธ์กับข้อมูลอื่นๆ เป็นต้น จึงต้องมีการแทนที่ การปรับปรุง หรือการลบข้อมูลที่ไม่ถูกต้องเหล่านี้ออกไป เพื่อให้ข้อมูลมีคุณภาพ
1. การจัดการข้อมูลที่หาย(missing values)
2. การจัดการ outlier
3. การ Scaler ข้อมูลให้อยู่ใน scale เดียวกัน
4. การจัดการ feature ที่ไม่มีผลต่อการ predict



In [ ]:
dataset = pd.concat(objs=[df_train,df_test], axis=0).reset_index(drop=True)
dataset.head()

In [ ]:
dataset.describe()

จากข้อมูลทางสถิติเบื่องต้นในแต่ละ feature ค่อนข้างแตกต่างกัน อยู่ใน range ที่แตกต่างกันมาก การ train model จะไม่สามารถ ถึงจุด optimize ได้ แต่ในที่นี้เราจะมาทำการ EDA แต่และจัดการข้อมูลให้ดีพร้อมในการเทรนโมเดลต่อไป

In [ ]:
dataset.info()

ดูข้อมูล missing values ที่หายไปในแต่ละ feature 

In [ ]:
dataset.isna().sum()

In [ ]:
dataset.shape,df_train.shape,df_test.shape

In [ ]:
df_train = df_train.fillna(0)
df_train.isna().sum()

จัดการ missing values ด้วยการให้ค่านั้นเป็น 0 หรือ ที่ต้องการแต่ในที่นี้ให้เป็น 0


In [ ]:
df_train.head()

**ปรับขนาดแต่ละคุณลักษณะด้วยค่าสัมบูรณ์สูงสุด(MaxAbsScaler)**

    ตัวประมาณนี้จะปรับขนาดและแปลคุณลักษณะแต่ละอย่างแยกกัน เพื่อให้ค่าสัมบูรณ์สูงสุดของคุณลักษณะแต่ละรายการในชุดการฝึกจะเป็น 1.0 ไม่เลื่อน/จัดศูนย์กลางข้อมูล จึงไม่ทำลายความกระจัดกระจายใดๆ โดยทำทั้ง Train , Test data set

In [ ]:
df_train_norm = df_train.copy()
#normalization dataset
for column in df_train_norm.columns:
    df_train_norm[column] = df_train_norm[column]  / df_train_norm[column].abs().max()
      
'''
for col in df_train.columns:
    df_train[col] = normalize(df_train,norm =)
'''
df_train_norm.head()

In [ ]:
df_test_norm = df_test.copy()
#normalization dataset
for column in df_test_norm.columns:
    df_test_norm[column] = df_test_norm[column]  / df_test_norm[column].abs().max()
      
'''
for col in df_train.columns:
    df_train[col] = normalize(df_train,norm =)
'''
df_test_norm.head()

In [ ]:
df_train.shape

**การจัดการ outlier**

    Outlier คือ data points ที่มีค่าสูงหรือต่ำกว่า data points ส่วนใหญ่ในชุดข้อมูลหนึ่งๆอย่างมาก เช่น ส่วนสูง 210 cm คือค่า outlier สำหรับคนไทย เพราะเราแทบไม่เคยเห็นคนไทยสูงเกิน 2 เมตรเลย เป็นต้น อ้างอิงสถิติจากเว็บไซต์ sizethailand ส่วนสูงเฉลี่ยชายและหญิงชาวไทย อายุ 16-25 ปี อยู่ที่ 171 cm และ 159 cm ตามลำดับ
    
    โดยในการจัดการข้อมูลชุดนี้จัดโดยวิธ๊ IQR
    ซึ่ง Outlier จะอยู่ที่ด้านปลายทั้งสองของ Distribution ใช้ค่า factor k โดยทั่วไปจะกำหนดไว้ที่ 1.5 ถึง 3 เท่าของค่า IQR โปรดที่ดูที่รูปด้านล่างประกอบ ที่ 1.5IQR คือพื้นที่ที่ยังคงเป็น Normally อยู่ทั้งนี้ 
    
    Lower Anomaly = Q1 - 1.5 x IQR
    Upper Anomaly = Q3 + 1.5 x IQR

In [ ]:
fig,ax = plt.subplots(figsize = (8,8))
sns.boxplot(data=df_train_norm,x='x1',color='orange')

boxplot แสดงให้เห็นข้อมูลที่อยู่นอกช่วง

In [ ]:
#IQR range
Q1 = df_train_norm.quantile(0.25)
Q3 = df_train_norm.quantile(0.75)
IQR = Q3 - Q1

outlier_step = IQR*1.5

#print(df_train_norm < (Q1 - 1.5 * IQR)) |(df_train_norm > (Q3 + 1.5 * IQR))        
df_out = df_train_norm[~((df_train_norm < (Q1 - outlier_step)) |(df_train_norm > (Q3 + outlier_step))).any(axis=1)].reset_index(drop=True)

In [ ]:
df_out.shape

หลังจากจัดการ outlier ข้อมูลหายไปจำนวนมาก ซึ่งอาจมีผลต่อการเทรนโมเดล จาก 7500 rows เหลือ 5179 rows ซึ่งหายไป 2000 กว่าๆ แถว 

In [ ]:
fig,ax = plt.subplots(figsize = (8,8))
sns.boxplot(data=df_out,x='x1',color='orange')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8))
sns.countplot(x="y", data=df_out, color="c")
plt.show()

In [ ]:
df_out['y'].value_counts()

จากการดู ข้อมูลเบื้องต้นพบว่าข้อมูลหายไปเยอะ และเกิด imbalance data ขึ้น ซึ่ง label ที่เป็น 0 มี 4014 และ ที่เป็น 1 มี 1165 ซึ่งข้อมูลไม่สมมาตรกัน

In [ ]:
df_out['y'].value_counts().plot(kind = 'pie',figsize=(8, 8),autopct='%1.1f%%',
        shadow=True, startangle=90,fontsize = 15)
plt.show()

เพื่อให้เห็นภาพบน Pie Chart ดังรูป

 ค่า **Pearson Correlation **
    
    เป็นวิธีทางสถิติที่นิยมนำมาใช้วิเคราะห์เพื่อหาความสัมพันธ์ของข้อมูลมากที่สุดวิธีหนึ่ง เนื่องจากเป็นวิธีที่เข้าใจง่ายและสามารถคำนวนได้ไม่ยาก โดยค่า Pearson Correlation จะมีค่าอยู่ระหว่าง -1.0 ถึง +1.0 ซึ่งหากมีค่าใกล้ -1.0 นั้นหมายความว่าตัวแปรทั้งสองตัวมีความสัมพันธ์กันอย่างมากในเชิงตรงกันข้าม หากมีค่าใกล้ +1.0 นั้นหมายความว่า ตัวแปรทั้งสองมีความสัมพันธ์กันอย่างมากในทิศทางเดียวกัน และหากมีค่าเป็น 0 นั้นหมายความว่า ตัวแปรทั้งสองตัวไม่มีความสัมพันธ์ต่อกัน

In [ ]:
fig,ax = plt.subplots(figsize = (14,12))
plt.title('Pearson Correlation of Features',loc = 'center',fontsize= 14)
sns.heatmap(df_out.corr(),linewidths=0.1,vmax=1.0,vmin = -1.0,
            square=True, linecolor='white', annot=True)
plt.show()

จากภาพข้อมูล Pearson Correlation เพื่อแสดงความสัมพันธ์ของแต่ละ feature  ค่า x แต่ละค่ากับ y 

In [ ]:
x_train = df_out.drop(columns='y')
y_train = df_out['y']
x_train.head()

In [ ]:
df_test_norm.head()

การจัดการข้อมูลที่หายไปโดยใช้ธี Oversampling 

Oversampling โดยใช้ SMOTE
    SMOTE (Synthetic Minority Oversampling Technique) เราจะทำการรวบรวม Elements สำหรับ Minority Class ใน Vicinity ของ Element ที่มีอยู่แล้ว โดยมีวิธีการดังนี้ คือ
     ซึ่งมีวิธีการคือ

* ระบุ feature vector และ จุดที่ใกล้ที่สุด
* ใช้ความแตกต่างระหว่างทั้งสอง
* คูณความแตกต่างด้วยตัวเลขสุ่มระหว่าง 0 ถึง 1
* ระบุจุดใหม่บนส่วนของเส้นตรงโดยการเพิ่มหมายเลขสุ่มลงใน feature vector
* ทำซ้ำขั้นตอนสำหรับ feature vector ที่ระบุ


![](http://images.techstarthailand.com/images/blog/Article2019/SamplingAlgorithms/SMOTE01.png)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

x_sm

# 3.Feature Selection

feature selection คือ การเลือก feature ที่สำคัญ และเกี่ยวข้องมากที่สุดจากชุดคุณสมบัติมากมายในชุดข้อมูลที่กำหนดซึ่งช่วยมีผลในการการเทรนโมเดลดงนี้
* เทรนโมเดลเร็วขึ้น
* ลดความซับซ้อนของโมเดล สามารถตีความได้ง่ายมากขึ้น
* ลดการ overfitting ที่อาจเกิดขึ้นได้


    การทำ ANOVA เป็นวิธีการหนึ่ง เพื่อหาความจัดการ feature ซึ่งหน้าที่ของ ANOVA นั้น มีไว้เพื่อทดสอบสมมติฐานทางสถิติของตัวแปรที่มากกว่า 2 มีความสัมพันธ์กันอย่างไรแบบ Univariate feature selection โดยการเลือก feature โดย f-core ซึ่งเป็นวิธีการทางสถิติทดสอบ มันสามารถเห็นได้ว่าเป็นขั้นตอนก่อนการประมวลผลไปยังตัวประมาณ ในที่นี้โดยตั้ง นัยสำคัญของข้อมูลที่ 0.05 โดยทดสอบสมมุติฐาน x1,x2,x3,... x20 มีผลต่อ y 

In [ ]:
BestFeatures = SelectKBest(score_func=f_classif, k=10)
fit = BestFeatures.fit(x_train,y_train)

df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(x_train.columns)

f_Scores = pd.concat([df_columns,df_scores],axis=1)              
f_Scores.columns = ['features','Score']  

f_Scores

In [ ]:
f_Scores[f_Scores['Score']<0.05]

แสดงให้เห็นว่า ค่า f-score ของ x1,x4,x20 มีค่าน้อยกว่า 0.05 ซึ่งจะทำการ feature นี้ออก เพื่อใช้ในการสร้าง train model ต่อไป


In [ ]:
x_train.drop(columns = ['x1','x4','x20'])
y_train.drop(columns = ['x1','x4','x20'])
df_test_norm.drop(columns = ['x1','x4','x20'])

# 4. Training and Evaluating Model 

Cross validate models 

โดยการเปรียบเทียบโมเดลที่ดังนี้โดยทำ cross validation โมเดลและดูผลเบื้องต้นเพื่อใช้ในการวิเคราะห์และเลือกโมเดลต่อไป

* SVC
* AdaBoost
* Random Forest
* Extra Trees
* Gradient Boosting
* KNN
* Logistic regression


In [ ]:
def model_cross_validation(x_train,y_train,random_state):
    kfold = StratifiedKFold(n_splits = 5)
    model_accuracy_result_cv = {"LogisticRegression":'',
                           "SVM":'',
                          "Randomforest":'',
                          "GradientBoosting":'',
                           "AdaBoostiong":'',
                           "ExtraTree":''}
    
    #model classification
    LG = LogisticRegression(random_state=random_state)
    SVM = SVC(random_state=random_state)
    RF = RandomForestClassifier(random_state=random_state)
    KNN = KNeighborsClassifier()
    GB = GradientBoostingClassifier(random_state=random_state)
    AB = AdaBoostClassifier(random_state=random_state,learning_rate=0.1)
    ET = ExtraTreesClassifier(random_state=random_state)
    ml_model = [LG,SVM,RF,KNN,GB,AB,ET]
    
    cv_results = []
    for ml in ml_model :
        cv_results.append(cross_val_score(ml,X = x_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=5))
    
    cv_means = []
    cv_std = []
    for result in cv_results:
        cv_means.append(result.mean())
        cv_std.append(result.std())
       

    cv_res = pd.DataFrame({"CV Means":cv_means,
                           "CV errors": cv_std,
                           "Algorithm":["LogisticRegression",
                                       "SVM",
                                       "RadomForest",
                                        "KNN",
                                       "GradientBoosting",
                                       "AdaBoosting",
                                       "ExtraTree"]})
    print(cv_res)
    g = sns.barplot("CV Means","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
    g.set_xlabel("CV Mean Accuracy")
    g = g.set_title("Cross Validation Evaluation")
    

In [ ]:
model_cross_validation(x_train = x_sm,y_train = y_sm,random_state = 42)

จากข้อูลการทำ Cross Validation พบว่า ค่าไม่แตกต่างกันมาก เกือบทุก model ทีค่า accuracy mean ไม่แตกต่างกันมากทำให้ยากที่จะประเมิน แต่จากข้อมูลเบิ้องต้นแต่ละโมเดลค่อยข้างมีค่า accuracy สูงอยู่ที่ประมาณ 0.95 เกือบทั้งหมด ่ต่อไปจะเป็นการ Hyperparameter Tuning เพื่อ Optimization model ให้ดีมากขึ้นพัฒนา Performance ของแต่ละ model แต่จากผล จะทำการ Tuning Model 



# 5.HyperParameter Tuning

การทำ Hyperparameter Tuning ในบางครั้งอาจจะถูกเรียกว่า Hyperparameter Optimization นับได้ว่าเป็น Optimization Problem รูปแบบหนึ่ง เนื่องจากเราต้องการหาว่า Set ของ Hyperparameters ที่เหมาะสมสำหรับโมเดลประเภทนั้น ๆ ที่จะส่งผลให้โมเดลมีความแม่นยำ (Accuracy) ที่สูง หรือต้องการลดค่า Loss ให้มีค่าต่ำที่สุด โดยทำการ Tuning ทุกโมเดล และ แสดงค่า accuracy ที่ดีที่สุดของแต่ละโมเดล เพื่อทำการเลือกโมเดลที่เหมาะสมที่สุด

In [ ]:
# Cross validate model with Kfold
kfold = StratifiedKFold(n_splits = 5)

In [ ]:
#AdaBoost 
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

#Search grid for optimal parameters
ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(x_sm,y_sm)

ada_best = gsadaDTC.best_estimator_

# Best score
gsadaDTC.best_score_

In [ ]:
#ExtraTrees 
ExtC = ExtraTreesClassifier()

#Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3,10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsExtC.fit(x_train,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [ ]:
#Support Vetor Machine
SVMC = SVC(probability=True)

#Search grid for optimal parameters
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}


gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(x_sm,y_sm)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

In [ ]:
# RandomForest
RFC = RandomForestClassifier()

#Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(x_sm,y_sm)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [ ]:
#KNeighborsNearest
KNN = KNeighborsClassifier()

#Search grid for optimal parameters
KNN_params_grid = {'n_neighbors':[6,8,10,12,14,16,18,20],
         'leaf_size':list(range(1,50,5))}

gsKNN = GridSearchCV(KNN, param_grid= KNN_params_grid, cv = kfold,scoring = "accuracy",verbose=1)

gsKNN.fit(x_sm,y_sm)

KNN_best = gsKNN.best_estimator_

# Best score
gsKNN.best_score_

In [ ]:
#Logistic Regression
lrr = LogisticRegression()

#Search grid for optimal parameters
lrr_params_grid ={"C":np.logspace(-3,3,7), 
      "penalty":["l1","l2"]}

gsLRR =GridSearchCV(lrr,param_grid=lrr_params_grid,cv=kfold)

gsLRR.fit(x_sm,y_sm)

LRR_best = gsLRR.best_estimator_

# Best score
gsLRR.best_score_

In [ ]:
# Gradientboosting

GBC = GradientBoostingClassifier()

#Search grid for optimal parameters
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(x_sm,y_sm)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

# 6.Learning Curve 

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [ ]:
g = plot_learning_curve(gsExtC.best_estimator_,"ExtraTrees learning curves",x_train,y_train,cv=kfold)

In [ ]:
g = plot_learning_curve(gsSVMC.best_estimator_,"SVC learning curves",x_train,y_train,cv=kfold)

In [ ]:
g = plot_learning_curve(gsadaDTC.best_estimator_,"AdaBoost learning curves",x_train,y_train,cv=kfold)

In [ ]:
g = plot_learning_curve(gsRFC.best_estimator_,"RF mearning curves",x_train,y_train,cv=kfold)

In [ ]:
g = plot_learning_curve(gsKNN.best_estimator_,"KNN mearning curves",x_train,y_train,cv=kfold)

In [ ]:
g = plot_learning_curve(gsLRR.best_estimator_,"LRR mearning curves",x_train,y_train,cv=kfold)

In [ ]:
g = plot_learning_curve(gsGBC.best_estimator_,"GBC mearning curves",x_train,y_train,cv=kfold)

GradientBoosting,RandomForest,SupportVectorMachine และ Adaboost มีแนวโน้มที่จะเกิด Overfitting แต่ cross validation ยังคงมีค่า accuracy สูงกว่า model อื่นดังรูปที่ CV line อยู่ต่ำกว่า training line ซึ่งหากมี training dataset ที่เพิ่มขึ่นอาจจะทำให้ถึงจุดที่ optimal ของ model ดังกราฟ 

KNN และ Logistic จะสรุปค่อยข้าง good fit กว่าโมเดลอื่น เพราะ เส้นทั้ง 2 วิ่งใกล้กันและเข้าหากัน

In [ ]:
corr = pd.concat([pd.Series(RFC_best.predict(df_test_norm), name="RF"),
                              pd.Series(ExtC_best.predict(df_test_norm), name="EXT"),
                              pd.Series(SVMC_best.predict(df_test_norm), name="SVC"),
                              pd.Series(KNN_best.predict(df_test_norm), name="KNN"),
                              pd.Series(LRR_best.predict(df_test_norm), name="LRR"),
                              pd.Series(ada_best.predict(df_test_norm), name="ADA"),
                              pd.Series(GBC_best.predict(df_test_norm), name="GBC")],axis=1)

plt.figure(figsize=(18,18))
sns.heatmap(corr.corr(),annot=True)
plt.show()

แต่ละ model เหมือนจะค่อนข้างคล้ายกันไม่แตกต่างกันมากจาก ข้อมูล pearson correlation ระหว่างโมเดลแต่ละตัว โดยเลือกโมเดล 4 ตัวที่ดีที่สุด มาทำการ ensemble model โดย ใช้ VotingClassification แบบ Soft โดยแต่ละ model จะทำนายป้ายกำกับคลาสตาม argmax ของผลรวมของความน่าจะเป็นที่คาดการณ์ไว้ โดยโมเดลที่ใช้โหวต คือ

1. ExtraTreeClassification
2. SupportVectorMachine
3. RandomForest
4. GradientBoosting

In [ ]:
#VotingClassification
votingC = VotingClassifier(estimators= [('extc',ExtC_best),
('svc', SVMC_best),('rfc',RFC_best),('gbc',GBC_best)],voting = 'soft')

votingC.fit(x_train, y_train)

votingC.predict(df_test_norm)

In [ ]:
columns = ['id','Expected']
predict_result = votingC.predict(df_test_norm)
predict_result = predict_result.astype(int)
df_predict = pd.DataFrame({
    "id":list(range(1,2501)),
    "Expected":predict_result
})

df_predict['Expected'].value_counts()


In [ ]:
df_predict.to_csv('submission.csv',index=False)